# Introduction to LLM Agents with LangChain

Welcome to the workshop on building LLM agents with LangChain!

**The goal** 

With this notebook you will familiarize yourself with the key concepts of an LLM agent. At the end, you will have all the code you need for your very own agent and you will be able to build custom tools for your own use-case. 

🌟 So ... let us begin!  

**The use case** 

Summer holidays are coming up and you still don't know where to go. Oh no! 

You decide to build a tool that helps you get information on holiday locations. For example, you would like to to find out how big a specific city is, what sights are there to see, how the weather there is, and you would like to get a drawing of that place, to get a first impression. Because who does not like art? 

You will implement this through a sentinent LLM agent, who has access to
* the wikipedia API,
* a weather API,
* can generate images by using a HuggingFace API. 

**The steps**

0. [Install dependencies to get the project up and running](#0)
1. [Familiarize yourself with the default wikipedia tool](#1)
        <ol type="a">
        <li>[Explore tool parameters](#1a)</li>
        <li>[Run tool and explore output](#1b)</li>
        </ol>
2. [Build your own tool: weather api](#2)
3. Agent exercises

# Install dependencies <a id='0'></a>

Compile the cell below to install the dependencies. Consider clearing the cell output so it does not clutter your notebook.

In [ ]:
!python3 -m venv venv         
!source venv/bin/activate     
!pip3 install -r helper_functions/requirements.txt

In [ ]:
# Update helper_functions.keys.py based on private bin link
from helper_functions.keys import WEATHER_KEY, OPENAI_KEY

## A Langchain default tool: the [Wikipedia tool](https://python.langchain.com/v0.1/docs/integrations/tools/wikipedia/) <a id='1'></a>

The cell below loads the full wikipedia tool. It makes an API call to Wikipedia using the ``WikipediaAPIWrapper`` and returns a summary of the queried article. ``WikipediaQueryRun`` then wraps this into a ready made tool.

In [ ]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

api_wrapper = WikipediaAPIWrapper(top_k_results=1)#, doc_content_chars_max=1000)
wiki_tool = WikipediaQueryRun(api_wrapper=api_wrapper)

#### Exercise 1 (a): Familiarize yourself with a default tool <a id='1a'></a>

Use the methods ``name``, ``description``, ``args``, ``return_direct``, ``metadata`` to familiarize yourself with the parameters of the tool. What is the meaning of the different parameters?

Background: each tool is a ``BaseTool`` class object, you can find its definition [here](https://api.python.langchain.com/en/latest/tools/langchain_core.tools.BaseTool.html#langchain_core.tools.BaseTool).

In [ ]:
print("Name: ", wiki_tool.name)

### TODO: insert your code here

#### Exercise 1 (b) run tool and explore output <a id='1b'></a>

* Use the ``.run(tool_input)`` method to execute the tool. The ``tool_input`` is the search term that you'd like to query wikipedia with.
* [Optional] Check out the arguments of the WikipediaAPIWrapper [here](https://api.python.langchain.com/en/latest/utilities/langchain_community.utilities.wikipedia.WikipediaAPIWrapper.html) and modify its parameters above. How does the output change? 

In [ ]:
# enter your code here

## Custom tools

You can build your own tools and don't have to rely on default tools. Tools can be built from any function with the LangChain class method ``StructuredTool.from_function()``(see [here](https://python.langchain.com/v0.1/docs/modules/tools/custom_tools/#structuredtool-dataclass)). The basic elements are
* The **function** you would like to be executed when the tool is called
* The definition of the **input parameters**
* The tool **description**

The tool description is especially important, since this is what the agent will use to make the deicion if this tool should be used.

Below you see the wikipedia tool, built from the basic elements described above:

In [ ]:
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import StructuredTool

# define the function
def wikipedia_caller(query:str) ->str:
    """This function queries wikipedia through a search query."""
    return api_wrapper.run(query)

# Input parameter definition
class QueryInput(BaseModel):
    query: str = Field(description="Input search query")

# the tool description
description: str = (
        "A wrapper around Wikipedia. "
        "Useful for when you need to answer general questions about "
        "people, places, companies, facts, historical events, or other subjects. "
        "Input should be a search query."
    )


# fuse the function, input parameters and description into a tool. 
my_own_wiki_tool = StructuredTool.from_function(
    func=wikipedia_caller,
    name="wikipedia",
    description=description,
    args_schema=QueryInput,
    return_direct=False,
)

# test the output of the tool
print(my_own_wiki_tool.run('pyladies'))

#### Exercise 2: Build your own tool: weather api <a id='2'></a>
The goal is to build a tool that extracts weather information from the weather site visualcrossing.com. You typically need an API key to extract information from a website. In this example we provide you with the API key. 

Task: 
- The tool function is already provided to your. Build the tool by defining the input parameters and the descriptions. 
- Turn function, description and input parameters into a tool through ``StructuredTool.from_function()``
- test if the tool gives an output

In [ ]:
import requests # for API calls

# define the function
def extract_city_weather(city:str)->str:

    # Build the API URL
    url = f"https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/{city}?key={WEATHER_KEY}&unitGroup=metric"

    response = requests.get(url)

    # extract response
    if response.status_code == 200:
        data = response.json()
        current_temp = data['days'][0]['temp']
        output = f"Current temperature in {city}: {current_temp}°C"
    else:
        output = f"Error: {response.status_code}"

    return output

# Input parameter definition
class WeatherInput(BaseModel):
    # insert your code here

# the tool description
description: str = (
        # TODO: insert your code here
    )

# fuse the function, input parameters and description into a tool. 
weather_tool = StructuredTool.from_function(
    # TODO: insert your code here
)

# test the output of the tool
print(weather_tool.run('Amsterdam'))

# TODO: try generating more ouputs

Combine the individual tools into a list. Your collection of tools is not ready to be used by an agent.

In [ ]:
tools = [my_own_wiki_tool, weather_tool]

## Agents

Agents combine the functionality of two components: LLMs and Tools. They empower an LLM to be able to execute additional tasks and reason through a problem. Namely, LLMs have knowlegde on data that was used at time of training. However, they lack knowledge about up-to-date happenings and information. They consist of the following components: 
- **LLM**: A pre-trained LLM.
- **List of tools**: List of tools that give additional functionality to the LLM.
- **Prompt**: A description/guideline that is given to the Agent to create "awareness" of its purpose.

One use case of LLM Agents is to make it possible to have LLMs with access to real time information, like the current weather. Namely, when a prompt is called, agents have an LLM and Tools at their disposal. Based on the prompt, they can deide what would be a better fit for the given prompt: information already available to the LLM, or information that can be fetched with an API (Tool). E.g. for a given prompt "What is the **usual** temperature in Amsterdam in summer?", an LLM will likely already have knowledge. However, a prompt "What is the **cureent** temperature in Amsterdam?", a weather API would be a better source of information, and in this case the Agent will decide to use the information from a weather tool. If such a tool is not available to the angent, the agent will repsond that the requested information is not available. 

So, basically, you can think of Agents as usual LLMs but with more "skills". Cool, right?

Let's see this through an example. 

In [ ]:
# dependencies
from langchain_openai import ChatOpenAI # call openAI as agent llm
from langchain import hub # for the prompt, we are going to skip this
from langchain.agents import create_tool_calling_agent # set up the agent
from langchain.agents import AgentExecutor # execute agent


from helper_functions.tools import weather_tool, my_own_wiki_tool, image_tool

In [ ]:
# Load Tools
tools = [my_own_wiki_tool, weather_tool, image_tool]

# Load LLM
llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0, api_key=OPENAI_KEY)

# Get the prompt to use - you can modify this! With this you let the agent know what its purpose is.
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages
print(type(prompt))

# Define  the agent (load the LLM and the list of tools)
agent = create_tool_calling_agent(llm = llm, tools = tools, prompt = prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

print("Your agent is ready.")


In the examples below, notice how the Agent is decides to use a different tool based on the context.

#### Question 1

In [ ]:
question_1 = "Where is Amsterdam?"


print(f"Question 1: {question_1}")
agent_executor.invoke({"input": question_1})

#### Question 2

In [ ]:
question_2 = "What is the cureent temperature in Amsterdam?"

print(f"Question 1: {question_2}")
agent_executor.invoke({"input": question_2})

#### Question 3

In [ ]:
question_3 = "What should I visit in Amsterdam? Show me an photo"

print(f"Question 1: {question_3}")
agent_executor.invoke({"input": question_3})

#### Exercise 3 (a): Build your own agent  <a id='2'></a>
The goal is to build an agent that uses the tools you previously developed. Feel free to also use the pre-made tools, available at `helper_functions/tools.py`

Task: 
- A template for defining an agent and an API key are  already provided to your. Build the agent by providing a list of tools, and an LLM `gpt-3.5-turbo-0125`.
- Test if the agent gives an output.
- Observe how the output changes if you provide less tools in your list of tools.

In [ ]:
# Load Tools
tools = [
# TODO: insert your code here
]

# Load LLM
llm = ChatOpenAI(model=
                 # TODO: insert your code here
                 temperature=0, 
                 api_key=OPENAI_KEY)

# Get the prompt to use - you can modify this! With this you let the agent know what its purpose is. But for now, let's keep it as is.
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages
print(type(prompt))

# Define the agent and agent executor (load the LLM, the list of tools, and the prompt (descripiton))

agent = create_tool_calling_agent(
    # TODO: insert your code here
    )
agent_executor = AgentExecutor(
    # TODO: insert your code here
)

print("Your agent is ready.")


#### Exercise 3 (b): Invoke as many tools as you can  <a id='2'></a>
Try various prompts to call the agent and follow the generated reasoning process in the response. The goal is to call the agent in a way thay it will use as many tools as possible. 

Let's see who can reach the highest number of tools used with a single prompt! 

In [ ]:
question = "
#TODO: Insert your code here
"

print(f"Question 1: {question}")
agent_executor.invoke({"input": question})

#### Exercise 3 (b): Optimize the agent prompt  <a id='2'></a>
So far we played around with the provided LLM and list of tools. Now let's look into the 3rd component: the Agent promot. 

Task:
- Modify the agent prompt and observe the difference in the output. 